In [13]:
import pandas as pd
import numpy as np
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
from collections import Counter
import pickle as pkl
import datetime


In [3]:
dedis = pd.read_csv('../files/dedis-3.csv', sep=',', names=['email', 'movie', 'date', 'stars'],
                    skipinitialspace=True)
imdb = pd.read_csv('../files/imdb-3.csv', sep=',', names=['email', 'movie', 'date', 'stars'], skipinitialspace=True)

# Finding my email hash

In [92]:
# def possible_dates(input_date, stars):
#     dates = []
#     for delta in range(-13,14,1):
#         new_date = (datetime.datetime.strptime(input_date, '%d/%m/%y') + datetime.timedelta(days=delta)).strftime('%d/%m/%y')
#         weight = 14 - abs(delta)
#         dates.append((new_date, stars, weight))
#     return dates

def possible_dates(input_date):
    dates = []
    for delta in range(-13,14,1):
        new_date = (datetime.datetime.strptime(input_date, '%d/%m/%y') + datetime.timedelta(days=delta)).strftime('%d/%m/%y')
        dates.append(new_date)
    return dates

def possible_dates_weights(input_date):
    weights = []
    for delta in range(-13,14,1):
        weights.append(14 - abs(delta))
    return weights

def dist_to_original_date(date, original_date):
    d2 = datetime.datetime.strptime(date, '%d/%m/%y')
    d1 = datetime.datetime.strptime(original_date, '%d/%m/%y')
    diff = abs((d2 - d1).days)
    return diff

In [93]:
dist_to_original_date('01/09/08', '10/09/08')

9

In [137]:
DF_correct_date = dedis[dedis['date'].isin(possible_dates('01/09/08'))]
DF_correct_date_stars = DF_correct_date.loc[DF_correct_date['stars']==3]
found_dates = DF_correct_date_stars['date'].values.tolist()
# date_tuples = list(zip(found_dates, len(found_dates)*['01/09/08']))
otiginal_date_repeated = len(found_dates)*['01/09/08']

distance_to_original_date = list(map(dist_to_original_date, found_dates, otiginal_date_repeated))
# distance_to_original_date
DF_correct_date_stars.assign(dist=distance_to_original_date).sort_values(by='dist', ascending=True).reset_index().loc[0:2,:]

,index,email,movie,date,stars,dist
0,374,9b18194fed64458396f54d2191adc7cf881709351ec02b...,ac2d5896f8fe48d65558105f9b3ef885c402e5ce8aae89...,01/09/08,3,0
1,894,3039e0120d7b7f568eaf794198f0a6bb3bf222f8b5138e...,e1baf949a8283986e822510050c23c21015e4c35617bd0...,02/09/08,3,1
2,895,d196ac8e17d20bad5c7f96bd6e947a4906ddf746e6a424...,c372501e5ec535eb180c5e853da36348d72704356ac710...,02/09/08,3,1


In [150]:
def find_matches_in_dedis(date, stars, slack):
    DF_correct_date = dedis[dedis['date'].isin(possible_dates(date))]
    DF_correct_date_stars = DF_correct_date.loc[DF_correct_date['stars']==stars]
    found_dates = DF_correct_date_stars['date'].values.tolist()
    otiginal_date_repeated = len(found_dates)*[date]

    distance_to_original_date = list(map(dist_to_original_date, found_dates, otiginal_date_repeated))
    return DF_correct_date_stars.assign(dist=distance_to_original_date).sort_values(by='dist', ascending=True).reset_index().loc[0:slack,:]

In [180]:
# find_matches_in_dedis('01/09/08', 3, 2)
# find_matches_in_dedis('02/05/02', 3, 2)
# find_matches_in_dedis('02/09/01', 2, 2)
# find_matches_in_dedis('03/06/04', 3, 2)

my_dates = (imdb.loc[imdb['email'] == 'mohammad.yaghini@epfl.ch'])['date'].values.tolist()
my_stars = (imdb.loc[imdb['email'] == 'mohammad.yaghini@epfl.ch'])['stars'].values.tolist()
my_slack = [2]*len(my_dates)

list_of_match_DFs = list(map(find_matches_in_dedis, my_dates, my_stars, my_slack))

my_possible_hashes = []
for DF in list_of_match_DFs:
    my_possible_hashes += DF['email'].values.tolist()
    
counted = Counter(my_possible_hashes)
counted_DF = pd.DataFrame(list(zip(list(counted.keys()), list(counted.values()))), columns=['hash', 'freq']).sort_values(by='freq', ascending = False).reset_index()
my_definite_hash = counted_DF.loc[0]['hash']
my_definite_hash

'63084100821c57736c52120612923610b5135f77651345d8cb5626a6936f4de2'

# Finding my movies

In [209]:
list_of_movies = list(set(imdb['movie'].values.tolist()))

movie2hash = {}
hash2movie = {}
for movie in list_of_movies:
#     movie = list_of_movies[2]
#     movie
    movie_dates = (imdb.loc[imdb['movie'] == movie])['date'].values.tolist()
    movie_stars = (imdb.loc[imdb['movie'] == movie])['stars'].values.tolist()
    movie_slack = [3]*len(movie_dates)

    list_of_movie_match_DFs = list(map(find_matches_in_dedis, movie_dates, movie_stars, movie_slack))
    # list_of_movie_match_DFs[0]
    movie_possible_hashes = []
    for DF in list_of_movie_match_DFs:
        movie_possible_hashes += DF['movie'].values.tolist()

    movie_counted = Counter(movie_possible_hashes)
    movie_counted_DF = pd.DataFrame(list(zip(list(movie_counted.keys()), list(movie_counted.values()))), columns=['hash', 'freq']).sort_values(by='freq', ascending = False).reset_index()
#     movie_counted_DF
    movie_definite_hash = movie_counted_DF.loc[0]['hash']
#     movie_definite_hash
    movie2hash[movie] = movie_definite_hash
    hash2movie[movie_definite_hash] = movie
    

In [210]:
len(list(set(dedis['movie'].values.tolist())))
len(hash2movie)

299

299

In [212]:
my_movies_hashed = dedis.loc[dedis['email'] == my_definite_hash]['movie'].values.tolist()
with open('ex1_3.txt', 'w') as f:
    for hash_ in my_movies_hashed:
        print(hash2movie[hash_], file=f)